# Mass-Radius Manifold Visualization

## Detailed Analysis

This script generates the primary visual summaries of the neutron star dataset in the Mass-Radius ($M-R$) plane. It produces "triptych" (three-panel) figures that compare the populations of Hadronic and Quark stars side-by-side and in superposition.

Two distinct visualization styles are implemented:
1.  **Statistical View (KDE):** Uses Kernel Density Estimation to show the probability density of stable star configurations. This highlights where the populations are most likely to reside.
2.  **Morphological View (Curves):** Plots the raw Equation of State (EoS) trajectories. This highlights the "flow" and topology of the curves, showing how individual families of stars evolve as central pressure increases.

## Physics and Math

### Mass-Radius Relation
The plots visualize the solution space of the Tolman-Oppenheimer-Volkoff (TOV) equations. Each point $(R, M)$ represents a stable hydrostatic equilibrium configuration.

### General Relativity Constraints
The script visualizes the **Buchdahl Limit**, which defines the maximum compactness allowed for a static, spherically symmetric fluid sphere in General Relativity. A star cannot exist if its radius is smaller than:

$$
R < \frac{9}{4} \frac{GM}{c^2}
$$

In the code, this boundary is drawn using the conversion factor `A_CONV` ($\approx 1.477$ km/$M_{\odot}$):

$$
R_{limit}(M) \approx 2.25 \times \text{A\_CONV} \times M
$$

### Kernel Density Estimation (KDE)
To visualize the statistical distribution of the populations, the script estimates the probability density function $P(R, M)$ using Gaussian kernels:

$$
\hat{P}(R, M) = \frac{1}{n} \sum_{i=1}^{n} K_h(R - R_i, M - M_i)
$$

where $(R_i, M_i)$ are the data points and $h$ is the smoothing bandwidth.

## Code Walkthrough

### 1. Data Preparation
The data is grouped by `Curve_ID` to handle entire EoS sequences. For the background line plots, a subset of curves is randomly selected to prevent visual clutter and reduce file size.

```python
# Sort by Density to ensure lines are drawn in physical order
g = group.sort_values(by='Eps_Central')
m_max = g['Mass'].max()
stable_g = g[g['Mass'] <= m_max] 
```
Only the stable branch (up to the maximum mass) is retained for the KDE calculation, as the unstable branch is astrophysically irrelevant for standard neutron stars.

### 2. KDE Calculation
The function `plot_physics_manifold` generates the statistical view. A grid is defined over the Mass-Radius plane (limits from `CONSTANTS`). The `scipy.stats.gaussian_kde` function computes the density of points on this grid.

```python
kernel = gaussian_kde(np.vstack([r_pts, m_pts]))
f = np.reshape(kernel(positions).T, xx.shape)

# Define levels relative to peak density
levels = [0.01 * f_max, 0.1 * f_max, 0.5 * f_max, f_max]
```
Contours are drawn at specific relative densities (e.g., 1%, 10%, 50% of the peak). Quark models are distinguished using a hatched pattern (`////`) to ensure legibility in black-and-white.

### 3. Curve Plotting
The function `plot_manifold_curves` plots the raw lines.
*   **Panel 1:** Hadronic curves (Green).
*   **Panel 2:** Quark curves (Magenta).
*   **Panel 3:** Both populations overlaid.

Transparency (`alpha`) is used to show density: regions where many lines overlap appear darker.

### 4. Constraints and Formatting
A helper function `_apply_common_formatting` ensures both plot styles share the same physical context.
*   **Buchdahl Wedge:** A gray shaded region indicates the zone forbidden by General Relativity.
*   **J0740 Limit:** A horizontal line at $2.08 M_{\odot}$ marks the mass of the massive pulsar PSR J0740+66, serving as a lower bound for the maximum mass capability of valid models.

## Visualization Output

The script saves two files to the `plots/` directory.

### 1. `fig_1_manifold_triptych.pdf` (Statistical KDE)
*   **Type:** Contour Plot.
*   **X-axis:** Radius $R$ [km].
*   **Y-axis:** Mass $M$ [$M_{\odot}$].
*   **Features:**
    *   **Panel (a):** Shows the Hadronic population is tightly constrained in radius (mostly 11-13 km).
    *   **Panel (b):** Shows the Quark population spans a much wider range of radii, extending to large radii at low masses.
    *   **Panel (c):** The intersection highlights the "Confusion Region" where the probabilities overlap significantly.

### 2. `fig_1_manifold_curves.pdf` (Morphological)
*   **Type:** Line Plot.
*   **Features:**
    *   Displays the full topological evolution of the stars.
    *   Hadronic curves typically curve backward (dR/dM < 0) as mass increases.
    *   Quark curves often slope forward (dR/dM > 0) at low masses before turning back, driven by vacuum pressure stability.